In [1]:
import torch
import torch.nn as nn
from peft import LoraConfig, get_peft_model, PeftModel

In [2]:
net1 = nn.Sequential(
  nn.Linear(10, 10),
  nn.ReLU(),
  nn.Linear(10, 2)
)

net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [3]:
for name, param in net1.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias


In [4]:
config1 = LoraConfig(target_modules=["0"])

In [5]:
model1 = get_peft_model(net1, config1)

model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [6]:
net2 = nn.Sequential(
  nn.Linear(10, 10),
  nn.ReLU(),
  nn.Linear(10, 2)
)

torch.save(net2.state_dict(), "/root/autodl-tmp/code/test-transformers/test-peft/peft/model_weights.pth")

net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [7]:
config1 = LoraConfig(target_modules=["0"])

In [8]:
config2 = LoraConfig(target_modules=["2"])

In [9]:
model1 = get_peft_model(net2, config1, adapter_name="loraA")

In [10]:
model1.add_adapter("loraB", config2)

In [11]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (loraB): Identity()
        )
        (lora_A): ModuleDict(
          (loraB): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraB): Linear(in_features=8, out_features=2, bias=False)
   

In [12]:
model1.active_adapter

'loraA'

In [13]:
model1(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.4800,  0.6054]], grad_fn=<AddmmBackward0>)

In [14]:
for name, param in model1.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        print(name)
        param.data = torch.ones_like(param)

base_model.model.0.lora_A.loraA.weight
base_model.model.0.lora_B.loraA.weight


In [15]:
model1(torch.arange(0, 10).view(1, 10).float())

tensor([[-289.4148,  111.2246]], grad_fn=<AddmmBackward0>)

In [16]:
model1.set_adapter("loraB")
model1.active_adapter

'loraB'

In [17]:
model1(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.4800,  0.6054]], grad_fn=<AddBackward0>)

In [18]:
model1.set_adapter("loraA")
model1.active_adapter

'loraA'

In [19]:
with model1.disable_adapter():
  print(model1(torch.arange(0, 10).view(1, 10).float()))
print(model1(torch.arange(0, 10).view(1, 10).float()))

tensor([[-1.4800,  0.6054]])
tensor([[-289.4148,  111.2246]], grad_fn=<AddmmBackward0>)


In [20]:
model1.save_pretrained("/root/autodl-tmp/code/test-transformers/test-peft/peft/models")

In [21]:
model1.save_pretrained("/root/autodl-tmp/code/test-transformers/test-peft/peft/loraA", selected_adapters=["loraA"])

In [22]:
net2 = nn.Sequential(
  nn.Linear(10, 10),
  nn.ReLU(),
  nn.Linear(10, 2)
)

net2.load_state_dict(torch.load("/root/autodl-tmp/code/test-transformers/test-peft/peft/model_weights.pth"))

<All keys matched successfully>

In [24]:
net2(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.4800,  0.6054]], grad_fn=<AddmmBackward0>)

In [26]:
model2 = PeftModel.from_pretrained(net2, model_id="/root/autodl-tmp/code/test-transformers/test-peft/peft/loraA/loraA", adapter_name="loraA")

In [27]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-289.4148,  111.2246]])